In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras
from chromalab.maxbasis import MaxBasis
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay
from chromalab.spectral_analysis import PCAAnalysis


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths1 = np.arange(400, 701, 1)
wavelengths5 = np.arange(400, 701, 5)
wavelengths10 = np.arange(400, 701, 10)

In [5]:
wavelengths = np.arange(400, 710, 10)
d65 = Illuminant.get("D65")

In [6]:
# natural_spectra = np.load("../../../Hyperspectral-Data/natural-spectra-400-700-5.npy")
tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=d65, verbose=True) 
maxbasis = MaxBasis(tetrachromat, verbose=True)

# viz = PSWrapper(tetrachromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.BOTH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True)

100%|██████████| 28/28 [00:00<00:00, 72.49it/s]
/home/jlee/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")


In [7]:
pca = PCAAnalysis(tetrachromat)
lmsq = pca.get_LMSQ_Responses()
np.save("lmsq.npy", lmsq)

100%|██████████| 9/9 [00:02<00:00,  3.02it/s]


In [ ]:
T = maxbasis.get_cone_to_maxbasis_transform()
responses = np.array([(viz.HMatrix@T@tetrachromat.observe(s[::2]))[1:] for s in natural_spectra])
spectras = convert_refs_to_spectras(natural_spectra, wavelengths5)
rgbs = np.array([s.to_rgb() for s in spectras])

In [ ]:
viz.renderObjectsPS(mesh_alpha=0.3, lattice_alpha=0.1)
viz.renderPointCloud(responses, rgbs, radius=0.01)
viz.renderQArrow(radius=0.005)

In [ ]:
viz.ps.show()

In [ ]:
dirname = "./output"
def rotate_once(offset, frame_count):
    for j in range(frame_count): # rotate once
        phi = 360 * j / frame_count
        point_3d = PSWrapper.polarToCartesian(3, 70, phi)
        viz.ps.look_at(point_3d, [0, 0, 0])
        viz.ps.screenshot(dirname + f"/frame_{offset * frame_count + j:03d}.png", True)
rot_per_sec = 0.5
frame_count = int(1/rot_per_sec * 30)
rotate_once(0, frame_count)

In [ ]:
exportAndPlay(dirname)